# Загрузка модели


In [1]:
import torch
from IPython.display import Audio

In [2]:
device = torch.device('cpu')
torch.set_num_threads(4)
local_file = 'v5_cis_base_nostress.jit'
torch.hub.download_url_to_file('https://models.silero.ai/models/tts/ru/v5_cis_base_nostress.jit',
                               local_file)
model = torch.jit.load(local_file)

100%|██████████| 87.4M/87.4M [00:06<00:00, 14.1MB/s]


Список дикторов

In [3]:
speakers = {'aze_gamat': 0,
            'bak_aigul': 1,
            'bak_alfia': 2,
            'bak_alfia2': 3,
            'bak_miyau': 4,
            'bak_ramilia': 5,
            'bel_anatoliy': 6,
            'bel_dmitriy': 7,
            'bel_larisa': 8,
            'chv_ekaterina': 9,
            'erz_alexandr': 10,
            'hye_zara': 11,
            'kat_vika': 12,
            'kaz_zhadyra': 13,
            'kaz_zhazira': 14,
            'kbd_eduard': 15,
            'kir_nurgul': 16,
            'kjh_karina': 17,
            'kjh_sibday': 18,
            'mdf_oksana': 19,
            'ru_aigul': 20,
            'ru_albina': 21,
            'ru_alexandr': 22,
            'ru_alfia': 23,
            'ru_alfia2': 24,
            'ru_bogdan': 25,
            'ru_dmitriy': 26,
            'ru_eduard': 27,
            'ru_ekaterina': 28,
            'ru_gamat': 29,
            'ru_igor': 30,
            'ru_karina': 31,
            'ru_kejilgan': 32,
            'ru_kermen': 33,
            'ru_marat': 34,
            'ru_miyau': 35,
            'ru_nurgul': 36,
            'ru_oksana': 37,
            'ru_onaoy': 38,
            'ru_ramilia': 39,
            'ru_roman': 40,
            'ru_safarhuja': 41,
            'ru_saida': 42,
            'ru_sibday': 43,
            'ru_vika': 44,
            'ru_zara': 45,
            'ru_zhadyra': 46,
            'ru_zhazira': 47,
            'ru_zinaida': 48,
            'sah_zinaida': 49,
            'tat_albina': 50,
            'tat_marat': 51,
            'tgk_onaoy': 52,
            'tgk_safarhuja': 53,
            'udm_bogdan': 54,
            'ukr_igor': 55,
            'ukr_roman': 56,
            'uzb_saida': 57,
            'xal_kejilgan': 58,
            'xal_kermen': 59}

Модель поддерживает только символы из этого набора:

In [4]:
symbols = "|||!'+,-.:;?hабвгдежзийклмнопрстуфхцчшщъыьэюяёєіїјўґғҕҗҙқҝҡңҥҫүұҳҷҹһӑӗәӝӟӣӥӧөӯӱӳӵӏ—… "

In [5]:
symbol_to_id = {s: i for i, s in enumerate(symbols)}

# Примеры работы

Все тексты нужно приводить к нижнему регистру и использовать только символы из набора "!'+,-.:;?hабвгдежзийклмнопрстуфхцчшщъыьэюяёєіїјўґғҕҗҙқҝҡңҥҫүұҳҷҹһӑӗәӝӟӣӥӧөӯӱӳӵӏ—… "

## Неславянские языки с алфавитом на кириллице

In [9]:
text = 'Мин китап уҡырға яратам.'.lower()

# переводим токены в тензор
text_ohe = torch.tensor([2] + [symbol_to_id[s] for s in text if s in symbol_to_id] + [1])
text_ohe = text_ohe[None, :]  # делаем массиву еще один dimension

# Выбираем спикера
speaker = 'bak_alfia'
speaker_id = torch.LongTensor([speakers[speaker]])

# Генерируем
audio = model(text_ohe,
              speaker_id)

Audio(audio.detach().numpy(), rate=48000)

## Славянские языки

В славянских языках необходимо ставить ударения

In [10]:
text = '+Я з р+аннього дит+инства д+уже любл+ю сл+ухати цік+аві к+азки.'.lower()

# переводим токены в тензор
text_ohe = torch.tensor([2] + [symbol_to_id[s] for s in text if s in symbol_to_id] + [1])
text_ohe = text_ohe[None, :]  # делаем массиву еще один dimension

# Выбираем спикера
speaker = 'ukr_roman'
speaker_id = torch.LongTensor([speakers[speaker]])

# Генерируем
audio = model(text_ohe,
              speaker_id)

Audio(audio.detach().numpy(), rate=48000)

## Языки с алфавитом, отличным от кириллицы (Азербайджанский/Армянский/Грузинский/Узбекский)

Все перечисленные языки переводятся в кириллицу через словарь

In [11]:
ext_alph = {'kat2ru': {'ა': 'а', 'ბ': 'б', 'გ': 'г', 'დ': 'д', 'ე': 'е', 'ვ': 'в', 'ზ': 'з', 'ჱ': 'эй', 'თ': 'т', 'ი': 'и', 'კ': "к'", 'ლ': 'л', 'მ': 'м', 'ნ': 'н', 'ჲ': 'йе', 'ო': 'о', 'პ': "п'", 'ჟ': 'ж', 'რ': 'р', 'ს': 'с', 'ტ': "т'", 'ჳ': 'уиэ', 'უ': 'у', 'ფ': 'п', 'ქ': 'к', 'ღ': 'г', 'ყ': "ҡ'", 'შ': 'ш', 'ჩ': 'ч', 'ც': 'ц', 'ძ': 'дз', 'წ': 'ць', 'ჭ': 'ч', 'ხ': 'х', 'ჴ': 'ҡ', 'ჯ': 'дж', 'ჰ': '-h-', 'ჵ': 'оо'},
            'aze2ru': {'A': 'А', 'B': 'Б', 'C': 'Ҹ', 'Ç': 'Ч', 'D': 'Д', 'E': 'Е', 'Ə': 'Ә', 'F': 'Ф', 'G': 'Ҝ', 'Ğ': 'Ғ', 'H': 'Һ', 'X': 'Х', 'I': 'Ы', 'İ': 'И', 'J': 'Ж', 'K': 'К', 'Q': 'Г', 'L': 'Л', 'M': 'М', 'N': 'Н', 'O': 'О', 'Ö': 'Ө', 'P': 'П', 'R': 'Р', 'S': 'С', 'Ş': 'Ш', 'T': 'Т', 'U': 'У', 'Ü': 'Ү', 'V': 'В', 'Y': 'Ј', 'Z': 'З', 'a': 'а', 'b': 'б', 'c': 'ҹ', 'ç': 'ч', 'd': 'д', 'e': 'е', 'ə': 'ә', 'f': 'ф', 'g': 'ҝ', 'ğ': 'ғ', 'h': 'һ', 'x': 'х', 'i': 'ы', 'i̇': 'и', 'j': 'ж', 'k': 'к', 'q': 'г', 'l': 'л', 'm': 'м', 'n': 'н', 'o': 'о', 'ö': 'ө', 'p': 'п', 'r': 'р', 's': 'с', 'ş': 'ш', 't': 'т', 'u': 'у', 'ü': 'ү', 'v': 'в', 'y': 'ј', 'z': 'з'},
            'arm2ru': {'Ա': 'А', 'Բ': 'Б', 'Գ': 'Г', 'Դ': 'Д', 'Ե': 'Ев', 'Զ': 'З', 'Է': 'Э', 'Ը': 'Ы', 'Թ': 'Т', 'Ժ': 'Ж', 'Ի': 'И', 'Լ': 'Л', 'Խ': 'Х', 'Ծ': 'Ц', 'Կ': 'К', 'Հ': 'Х', 'Ձ': 'Дз', 'Ղ': 'Х', 'Ճ': 'Ч', 'Մ': 'М', 'Յ': 'Й', 'Ն': 'Н', 'Շ': 'Ш', 'Ո': 'О', 'Չ': 'Ч', 'Պ': 'П', 'Ջ': 'Дж', 'Ռ': 'Р', 'Ս': 'С', 'Վ': 'В', 'Տ': 'Т', 'Ր': 'Р', 'Ց': 'Ц', 'Ւ': 'У', 'Փ': 'П', 'Ք': 'К', 'Օ': 'О', 'Ֆ': 'Ф', 'և': 'ев', 'ա': 'а', 'բ': 'б', 'գ': 'г', 'դ': 'д', 'ե': 'ев', 'զ': 'з', 'է': 'э', 'ը': 'ы', 'թ': 'т', 'ժ': 'ж', 'ի': 'и', 'լ': 'л', 'խ': 'х', 'ծ': 'ц', 'կ': 'к', 'հ': 'х', 'ձ': 'дз', 'ղ': 'х', 'ճ': 'ч', 'մ': 'м', 'յ': 'й', 'ն': 'н', 'շ': 'ш', 'ո': 'о', 'չ': 'ч', 'պ': 'п', 'ջ': 'дж', 'ռ': 'р', 'ս': 'с', 'վ': 'в', 'տ': 'т', 'ր': 'р', 'ց': 'ц', 'ւ': 'у', 'փ': 'п', 'ք': 'к', 'օ': 'о', 'ֆ': 'ф'},
            'uzb2ru': {'a': 'а', 'b': 'б', 'd': 'д', 'e': 'э', 'f': 'ф', 'g': 'г', 'h': 'ҳ', 'i': 'и', 'j': 'ж', 'k': 'к', 'l': 'л', 'm': 'м', 'n': 'н', 'o': 'о', 'p': 'п', 'q': 'қ', 'r': 'р', 's': 'с', 't': 'т', 'u': 'у', 'v': 'в', 'x': 'х', 'y': 'й', 'z': 'з', 'oʻ': 'ў', 'gʻ': 'ғ', 'sh': 'ш', 'ch': 'ч', 'ng': 'нг', '\'': 'ъ'}}

In [12]:
text = 'მე მომწონს წიგნების კითხვა.'.lower()

# переводим в кириллицу, используя грузинский словарь(kat2ru)
lang_dict = ext_alph['kat2ru']
text = [lang_dict.get(s, s) for s in text]

# переводим токены в тензор
text_ohe = torch.tensor([2] + [symbol_to_id[s] for s in text if s in symbol_to_id] + [1])
text_ohe = text_ohe[None, :]  # делаем массиву еще один dimension

# Выбираем спикера
speaker = 'kat_vika'
speaker_id = torch.LongTensor([speakers[speaker]])

# Генерируем
audio = model(text_ohe,
              speaker_id)

Audio(audio.detach().numpy(), rate=48000)